# 综合运用各种组件

Tensor对象、Graph对象、Op、Variable对象、占位符、Session对象、名称作用域。

In [1]:
import tensorflow as tf
import numpy as np

In [4]:
graph = tf.Graph()
with graph.as_default():
    
    with tf.name_scope("Variables"):
        # 数据流图的运行次数
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name="global_step")
        # 该模型所有输出随时间的累加
        total_output = tf.Variable(0.0, dtype=tf.float32, trainable = False, name="total_output")
    
    with tf.name_scope("transformation"):
        #
        with tf.name_scope("input"):
            #
            a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
        #    
        with tf.name_scope("intermediate_layer"):
            # 
            b = tf.reduce_prod(a, name="product_b")
            c = tf.reduce_sum(a, name="sum_c")
        #
        with tf.name_scope("output"):
            output = tf.add(b, c, name="output")
        
    with tf.name_scope("update"):
        update_total = total_output.assign_add(output)
        increament_step = global_step.assign_add(1)
        
    with tf.name_scope("summaries"):
        avg = tf.div(update_total, tf.cast(increament_step, tf.float32), name="average")
        
        #
        tf.summary.scalar('Output', output)
        tf.summary.scalar('Sum_of_outputs_over_time', update_total)
        tf.summary.scalar('Average_of_outputs_over_time', avg)
        
    with tf.name_scope("global_ops"):
        # 
        init = tf.global_variables_initializer()
        #
        merged_summaries = tf.summary.merge_all()

In [5]:
sess = tf.Session(graph=graph)
writer = tf.summary.FileWriter('./summary/improved_graph', graph)

sess.run(init)

In [6]:
def run_graph(input_tensor):
    feed_dict = {a: input_tensor}
    _, step, summary = sess.run([output, increament_step, merged_summaries], feed_dict=feed_dict)
    writer.add_summary(summary, global_step=step)

In [7]:
run_graph([2, 8])
run_graph([3, 1, 1, 3])
run_graph([8])
run_graph([1, 2, 3])
run_graph([11, 4])
run_graph([4, 1])
run_graph([7, 3, 1])
run_graph([6, 3])
run_graph([0, 2])
run_graph([5, 4, 6])

In [8]:
writer.flush()

In [9]:
writer.close()
sess.close()